In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from gerar_cpfs import gerador_de_cpfs

import time

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('./data/bacen-remuneracao.csv', encoding='unicode_escape', sep=';')

In [10]:
df.columns

Index(['ANO', 'MES', 'Id_SERVIDOR_PORTAL', 'CPF', 'NOME',
       'REMUNERAÇÃO BÁSICA BRUTA (R$)', 'REMUNERAÇÃO BÁSICA BRUTA (U$)',
       'ABATE-TETO (R$)', 'ABATE-TETO (U$)', 'GRATIFICAÇÃO NATALINA (R$)',
       'GRATIFICAÇÃO NATALINA (U$)',
       'ABATE-TETO DA GRATIFICAÇÃO NATALINA (R$)',
       'ABATE-TETO DA GRATIFICAÇÃO NATALINA (U$)', 'FÉRIAS (R$)',
       'FÉRIAS (U$)', 'OUTRAS REMUNERAÇÕES EVENTUAIS (R$)',
       'OUTRAS REMUNERAÇÕES EVENTUAIS (U$)', 'IRRF (R$)', 'IRRF (U$)',
       'PSS/RPGS (R$)', 'PSS/RPGS (U$)', 'DEMAIS DEDUÇÕES (R$)',
       'DEMAIS DEDUÇÕES (U$)', 'PENSÃO MILITAR (R$)', 'PENSÃO MILITAR (U$)',
       'FUNDO DE SAÚDE (R$)', 'FUNDO DE SAÚDE (U$)',
       'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (R$)',
       'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (U$)',
       'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$)',
       'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$)',
       'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*)',
       'VERBAS INDENIZATÓ

In [11]:
df = df.dropna(subset=['CPF', 'NOME'])

In [12]:
df.CPF

0       ***.161.491-**
1       ***.297.631-**
2       ***.895.388-**
3       ***.983.931-**
4       ***.210.784-**
             ...      
5383    ***.533.991-**
5384    ***.454.201-**
5385    ***.828.526-**
5386    ***.113.787-**
5387    ***.805.121-**
Name: CPF, Length: 5388, dtype: object

In [13]:
df['mid_cpf'] = df['CPF'].str.replace('\W', '')

C:\Users\karin\AppData\Local\Temp\ipykernel_6760\1078522682.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['mid_cpf'] = df['CPF'].str.replace('\W', '')
C:\Users\karin\AppData\Local\Temp\ipykernel_6760\1078522682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mid_cpf'] = df['CPF'].str.replace('\W', '')


In [15]:
df_cpf = df[['mid_cpf', 'NOME']]

In [16]:
df_cpf

,mid_cpf,NOME
0,161491,ABADIA MARCIA CUNHA
1,297631,ABDIAS CASSIO MARROCOS GOMES
2,895388,ABDIEL ANDRIOLO DE ANDRADE
3,983931,ABDON LOPES DE MENEZES
4,210784,ABEL AMBROSIO DA SILVA FILHO
...,...,...
5383,533991,ZULEICA DE MELO TELES
5384,454201,ZULEICA MARIA SOUZA PORTO
5385,828526,ZULMA GONCALVES DIAS
5386,113787,ZULMAR MAYETA


In [17]:
df_cpf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5388 entries, 0 to 5387
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   mid_cpf  5388 non-null   object
 1   NOME     5388 non-null   object
dtypes: object(2)
memory usage: 126.3+ KB


## Gerando CPFs

In [39]:
gerador = gerador_de_cpfs(nums_meio = 458763)
cpfs = gerador.gerar_cpfs()

In [18]:
from selenium import webdriver

DRIVER_PATH = '/path/to/chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.get('https://portaldatransparencia.gov.br/servidores/consulta?ordenarPor=nome&direcao=asc')

C:\Users\karin\AppData\Local\Temp\ipykernel_6760\2638167608.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH)


In [32]:
cpfs_inteiros = []
for i in range(len(df_cpf)):
    cpf_atual = df_cpf.mid_cpf[i]
    nome_atual = df_cpf.NOME[i]
    print(cpf_atual)
    print(nome_atual)

    # gerar cpfs inteiros
    gerador = gerador_de_cpfs(nums_meio = cpf_atual)
    cpfs = gerador.gerar_cpfs()

    for cpf in cpfs:
        print(cpf)
        
        # clicar no botão do cpf
        btn_cpf = driver.find_elements(By.CLASS_NAME, "box-tabela-filtro__gaveta")
        btn_cpf[1].click()

        # inserir cpf no campo de input
        insert_text = driver.find_element(By.ID, "cpf")
        insert_text.clear()
        insert_text.send_keys(cpf)

        time.sleep(0.3)

        # clicar no botão de adicionar
        btn_adicionar = driver.find_elements(By.CLASS_NAME, "btn-adicionar.btn-gaveta-consultar")[1]
        btn_adicionar.click()

        time.sleep(0.3)

        # clicar no botão de consultar
        btn_consultar = driver.find_element(By.CLASS_NAME, "btn-consultar.btn-filtros-aplicados-consultar")
        btn_consultar.click()

        # esperar carregar
        time.sleep(1)

        # verificar resposta da consulta
        try:
            table_element = driver.find_element(By.CLASS_NAME, "dataTables_empty")
            print(driver.find_element(By.CLASS_NAME, "dataTables_empty").text)
        except:
            nomes = driver.find_elements(By.CLASS_NAME, "coluna-livre")
            if nomes[10].text == nome_atual:
                print("CPF encontrado")
                cpfs_inteiros.append(cpf)
                break
            

161491
ABADIA MARCIA CUNHA
00016149165
Nenhum registro encontrado
00116149191
Nenhum registro encontrado
00216149118
Nenhum registro encontrado
00316149144
Nenhum registro encontrado
00416149170
Nenhum registro encontrado
00516149105
Nenhum registro encontrado
00616149123
Nenhum registro encontrado
00716149150
Nenhum registro encontrado
00816149186
Nenhum registro encontrado
00916149102
Nenhum registro encontrado
01016149182
Nenhum registro encontrado
01116149109
Nenhum registro encontrado
01216149135
Nenhum registro encontrado
01316149161
Nenhum registro encontrado
01416149198
Nenhum registro encontrado
01516149114
Nenhum registro encontrado
01616149140
Nenhum registro encontrado
01716149177
Nenhum registro encontrado
01816149101
Nenhum registro encontrado
01916149120
Nenhum registro encontrado
02016149108
Nenhum registro encontrado
02116149126
Nenhum registro encontrado
02216149152
Nenhum registro encontrado
02316149189
Nenhum registro encontrado
02416149105
Nenhum registro encontrad

KeyboardInterrupt: 

In [33]:
cpfs_inteiros

['18516149153', '22729763104']